### 6. Случайная величина имеет распределениe Парето


\begin{equation*} 
p(x) =
\begin{cases} 
\frac{\theta - 1}{x^{\theta}}, & \text{если } x \geq 1, \\
0, & \text{если } x < 1.
\end{cases}
\end{equation*} 


#### d)
Сгенерируйте выборку объема n = 100 для некоторого значения параметра $\theta$. Вычислите указанные выше доверительные интервалы для доверительной вероятности 0.95

#### t)
Численно постройте бутстраповский доверительный интервал двумя способами, используя параметрический бутстрап и непараметрический бутстрап.

#### f)
Сравнить все интервалы.


In [22]:
import numpy as np
import random

theta = 3

def p(x: float) -> float:
    return (theta - 1) / x ** theta if x >= 0 else 0

def F(x: float) -> float:
    return (1 - x ** (1 - theta)) if x >= 0 else 0

def F_inv(y: float) -> float:
    return (1 - y) ** (1 / (1 - theta))

n = 100
arr : list = [float(F_inv(random.random())) for _ in range(n)]

arr

[2.3173095175997567,
 2.1613144059653577,
 1.5099918273772706,
 1.0771726602597913,
 1.7649078317436646,
 1.6493767080737023,
 1.017041399107547,
 1.6240976687005857,
 1.0554178042683031,
 1.413082296769121,
 1.0576914333981962,
 1.5240679377342834,
 2.819064848589002,
 1.1332307908251937,
 1.2274744179136292,
 1.0999899232899004,
 1.4800062279632076,
 81.42405514929881,
 1.6809900635528727,
 1.5436143430111842,
 5.269700991188296,
 1.8640142479176807,
 4.017591038730939,
 2.128563524460023,
 1.3488840181287285,
 1.083973270879957,
 2.8325579084646093,
 1.2964789525377336,
 2.5761330110967355,
 2.6983287295370477,
 1.158319535345364,
 1.076216231148458,
 1.0185492574978272,
 1.2063660591007819,
 13.401681576025053,
 2.384256893981903,
 1.1623950738655449,
 1.2076477315006593,
 1.2538445459521161,
 1.1873360531502817,
 2.0213654019326035,
 1.1662021584964166,
 1.190375720103907,
 1.0483800968866215,
 14.571771896881545,
 1.4303078188394298,
 1.285196364327881,
 2.345796447554133,
 1.052

$$
\tilde{\theta} = 1 + \frac{1}{\overline{ln(x)}}
$$

$$
P(\tilde{\theta} - \frac{(\tilde{\theta} - 1)t_2}{\sqrt{n}} < \theta < \tilde{\theta} - \frac{(\tilde{\theta} - 1)t_1}{\sqrt{n}}) = \beta
$$

In [23]:
t_1 = -1.96
t_2 = 1.96


beta = 0.95

theta_wave = 1 + 1 / np.mean([np.log(x) for x in arr])

left_part = theta_wave - ((theta_wave - 1) * t_2) / np.sqrt(n)
right_part = theta_wave -  ((theta_wave - 1) * t_1) / np.sqrt(n)
print(f'Доверительный интервал: ({left_part}, {right_part})')
print(f'Длина интервала: {right_part - left_part}')


Доверительный интервал: (2.551509804949261, 3.3079673217901937)
Длина интервала: 0.7564575168409329


### Доверительный интервал для медианы

$$
P(g(\tilde{\theta}) - \frac{t_2 \cdot ln2 \cdot 2 ^ {\frac{1}{(\tilde{\theta} - 1)}}}{(\tilde{\theta} - 1)\sqrt{n}} < g(\theta) < g(\tilde{\theta}) - \frac{t_1 \cdot ln2 \cdot 2 ^ {\frac{1}{(\tilde{\theta} - 1)}}}{(\tilde{\theta} - 1)\sqrt{n}}) = \beta
$$

In [30]:

def g(theta : float) -> float:
    return 2 ** (1 / (theta - 1))

left_part = g(theta_wave) - (t_2 * np.log(2) * g(theta_wave)) / ((theta_wave - 1) * np.sqrt(n)) 
right_part = g(theta_wave) - (t_1 * np.log(2) * g(theta_wave)) / ((theta_wave - 1) * np.sqrt(n)) 
print(f'Доверительный интервал: ({left_part}, {right_part})')
print(f'Длина интервала: {right_part - left_part}')

print("Медиана: ", (sorted(arr)[49] + sorted(arr)[50]) / 2)

Доверительный интервал: (1.4126652521681857, 1.4491849400200805)
Длина интервала: 0.036519687851894744
Медиана:  1.4275993425786493


#### t)
Численно постройте бутстраповский доверительный интервал двумя способами, используя параметрический бутстрап и непараметрический бутстрап.

#### Непараметрический:


In [24]:
bootstrap_iteration = 1000

theta_wave = 1 + 1 / np.mean([np.log(x) for x in arr]) # Оценка ОМП

bootstrap_delta = []

for i in range(bootstrap_iteration):
    bootstrap_arr = np.random.choice(arr, size=len(arr), replace=True)
    bootstrap_theta = 1 + 1 / np.mean([np.log(x) for x in bootstrap_arr])
    bootstrap_delta.append(bootstrap_theta - theta_wave)

bootstrap_delta.sort()
t_1 = bootstrap_delta[int(bootstrap_iteration * (1 - beta) / 2)]
t_2 = bootstrap_delta[int(bootstrap_iteration * (1 + beta) / 2)]
right_part = -(t_1 - theta_wave)
left_part = -(t_2 - theta_wave)
print(f'Доверительный интервал: ({left_part}, {right_part})')
print(f'Длина интервала: {right_part - left_part}')



Доверительный интервал: (2.4246310039545422, 3.3279976637775066)
Длина интервала: 0.9033666598229644


#Параметрический:

In [25]:
def F_inv_star(y: float, theta_w: float) -> float:
    return (1 - y) ** (1 / (1 - theta_w))

bootstrap_iteration = 50000


for i in range(bootstrap_iteration):
    bootstrap_arr = [F_inv_star(random.random(), theta_wave) for _ in range(n)]
    bootstrap_theta = 1 + 1 / np.mean([np.log(x) for x in bootstrap_arr])
    bootstrap_delta.append(bootstrap_theta - theta_wave)

bootstrap_delta.sort()
t_1 = bootstrap_delta[int(bootstrap_iteration * (1 - beta) / 2)]
t_2 = bootstrap_delta[int(bootstrap_iteration * (1 + beta) / 2)]
right_part = -(t_1 - theta_wave)
left_part = -(t_2 - theta_wave)
print(f'Доверительный интервал: ({left_part}, {right_part})')
print(f'Длина интервала: {right_part - left_part}')


Доверительный интервал: (2.550540732817705, 3.2604532757331874)
Длина интервала: 0.7099125429154824


#### f)
Сравнить все интервалы.

По размеру интервала получаюется, что параметрический бутстрап < асимптотический метод < непараметрический бутстрап

Но так, как парметрический бутстрап может занижать дисперсию, то наверняка сказать, что он лучше нельзя.
